In [1]:
# rm(list=ls())
require(data.table)
require(MASS)
require(ggplot2)
require(gridExtra)
require(stringr)
require(doParallel)
require(abind)
require(Matrix)
set.seed(123)

## Resizing notebook plot space
options(repr.plot.width=16, repr.plot.height=9)

Loading required package: data.table
Loading required package: MASS
Loading required package: ggplot2
Loading required package: gridExtra
Loading required package: stringr
Loading required package: doParallel
Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Loading required package: abind
Loading required package: Matrix


## (0) Some functions

In [2]:
## Load in the R functions
source("/home/j/temp/central_comp/libraries/current/r/get_draws.R")

In [3]:
## Neal's multivariate copula function
draw2Dcopula <- function(X, cor_mat, df_return = F){
    
    ## If X is a df, convert to array; else continue
    
  L <- dim(X)[2]
  D <- dim(X)[3]
  Xsum <- apply(X, c(2, 3), sum)
  mvdat <- mvrnorm(n=D, mu=0 * 1:L, Sigma=cor_mat, empirical=TRUE)
  ranks <- apply(mvdat, 2, rank, ties.method="first")
  sortedXsim <- apply(Xsum, 1, function(x) sort(x, index.return=TRUE)$ix)
  sortedX <- X
  for(i in 1:L){
    sortedX[,i,] <- X[,i,sortedXsim[,i]]
  }
  Xcorr <- sortedX
  for(i in 1:L){
    Xcorr[,i,] <- sortedX[,i,ranks[,i]]
  }
  if (df_return==T) {
    return(data.table(melt(Xcorr)))
    }
      else {
          Xcorr
      }
}


In [4]:
## Stacking function: stack the names of the ones that matter and ones that don't
stacking_df <- function(data, to_stack,no_stack) {
    
    to_stack_name <- paste(to_stack, collapse="__")
    no_stack_name <- paste(no_stack, collapse="__")
    
    data[,paste0(to_stack_name) := do.call(paste, c(data[, .SD, .SDcols = to_stack], sep = "__")) ]
    data[, paste0(no_stack_name) := do.call(paste, c(data[, .SD, .SDcols = no_stack], sep = "__"))]
    
    return(data)
}

In [3]:
## Unstacking function: le opposite
unstack_df <- function(data, corr_unstack, no_corr_unstack, draw_name = "draw_num", data_col_name = "data",
                      wide_on_draws = F) {
    
    ## Input needs to be long on errything
    
    ## Split the variables according to the length of the corring_over and not_corring_over vectors
    ### This is much faster than explicitly defining the LHS var names because we're using 'set'

    cols_of_data <- ncol(data)    
    dalys_binded_long_splitters <- data[,1:cols_of_data]
    
    ##### NOTE: Why is it that I have to define the columns of 'data', or else it just doesn't work?? ####
    
    setDT(dalys_binded_long_splitters)[, paste0("not_corr", 1:length(no_corr_unstack)) := 
                                       tstrsplit(get(var_melters[1]), "__", type.convert = TRUE, fixed = TRUE)]
    setDT(dalys_binded_long_splitters)[, paste0("to_corr", 1:length(corr_unstack)) := 
                                       tstrsplit(get(var_melters[2]), "__", type.convert = TRUE, fixed = TRUE)]


    ## Keep what we need:
    dalys_keeperz <- dalys_binded_long_splitters[, .SD, 
                                                 .SDcols = c(paste0("not_corr", 1:length(no_corr_unstack)),
                                                             paste0("to_corr", 1:length(corr_unstack)),
                                                             "draw_num", "data")]
    
    ## Cast the draws out wide?
    if(wide_on_draws) {
        dalys_keeperz <- dcast(dalys_keeperz, )
    }
    
    ## Rename columns back
    for(i in 1:length(c(no_corr_unstack, corr_unstack))) {
        colnames(dalys_keeperz)[i] <- c(no_corr_unstack, corr_unstack)[i] 
    }
    return(dalys_keeperz)
}

## (1) Prepping the data to be into the array we want

In [5]:
## Get some example draws (causes 587 and 495)
dalys_diabetes <- get_draws(gbd_id_field = "cause_id", gbd_id = 587, source = 'dalynator', metric_id = 1,
                            measure_ids = 2, location_id = c(6,7,8), 
                            age_group_ids = c(6,7,8,9,10), sex_ids = c(1,2))

dalys_ihd <- get_draws(gbd_id_field = "cause_id", gbd_id = 495, source = 'dalynator', metric_id = 1,
                            measure_ids = 2, location_id = c(6,7,8), 
                            age_group_ids = c(6,7,8,9,10), sex_ids = c(1,2))

In [ ]:
head(dalys_ihd)

In [7]:
## Clean up and only what we need (loc, year, age, sex, draws):
dalys_ihd <- dalys_ihd[, .SD, .SDcols = c("location_id", "year_id", "age_group_id", "sex_id", "cause_id", paste0("draw_",c(0:999)) )]
dalys_diabetes <- dalys_diabetes[, .SD, .SDcols = c("location_id", "year_id", "age_group_id", "sex_id", "cause_id", paste0("draw_",c(0:999)) )]

In [8]:
## Convert age group to a string with leading zeroes, so that the sorting is maintained in the arrays
dalys_ihd[, age_group_id:= formatC(age_group_id, width = 3, format = "d", flag = "0")]
dalys_diabetes[, age_group_id:= formatC(age_group_id, width = 3, format = "d", flag = "0")]

In [9]:
## Stack age_sex and loc_year for each of the data tables by the variable we want to correlate over or not
corring_over <- c("age_group_id", "sex_id", "cause_id")
not_corring_over <- c("location_id", "year_id")

dalys_ihd <- stacking_df(dalys_ihd, to_stack =  corring_over, no_stack = not_corring_over)
dalys_diabetes <- stacking_df(dalys_diabetes, to_stack =  corring_over, no_stack = not_corring_over)

In [10]:
head(dalys_diabetes); tail(dalys_ihd)

location_id,year_id,age_group_id,sex_id,cause_id,draw_0,draw_1,draw_2,draw_3,draw_4,⋯,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,age_group_id__sex_id__cause_id,location_id__year_id
6,1990,006,1,587,5551.28707,5011.35001,5230.27951,5159.20116,4929.13923,⋯,4998.58775,5237.57671,5637.10652,5552.47410,5192.48345,4791.53376,5112.86431,4748.82328,006__1__587,6__1990
7,1990,006,1,587,35.05237,39.75230,22.31576,31.36539,21.13766,⋯,28.35600,51.33843,35.90585,40.73370,40.78756,47.48124,52.00294,45.74394,006__1__587,7__1990
8,1990,006,1,587,85.09490,84.98729,72.09881,107.28607,79.32551,⋯,55.39497,76.87085,51.15128,58.55369,47.29480,42.98546,57.92655,39.45834,006__1__587,8__1990
6,1990,007,1,587,15998.24064,11584.10129,11630.16977,12598.39286,10725.20921,⋯,10528.19276,11964.77253,11981.37321,11900.08308,10474.10660,12320.29350,10956.66726,9727.47450,007__1__587,6__1990
7,1990,007,1,587,113.51948,89.13242,108.93392,124.96546,93.38775,⋯,84.91567,120.01612,82.66760,119.78845,103.57053,106.14606,109.97217,88.41428,007__1__587,7__1990
8,1990,007,1,587,219.56580,144.88023,268.23797,179.55307,163.26509,⋯,238.10002,172.62521,157.83982,173.48822,156.35254,192.37304,157.26231,126.48048,007__1__587,8__1990


location_id,year_id,age_group_id,sex_id,cause_id,draw_0,draw_1,draw_2,draw_3,draw_4,⋯,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,age_group_id__sex_id__cause_id,location_id__year_id
6,2016,009,2,495,27897.3516,20812.0221,20069.5434,16749.4740,16041.2950,⋯,26350.1724,19655.6477,28564.5037,24999.9394,14590.0069,21843.9686,19873.9897,25697.2138,009__2__495,6__2016
7,2016,009,2,495,481.9214,570.2783,526.6523,322.3324,467.0658,⋯,744.0661,507.4481,469.7620,624.8477,434.9746,357.6581,674.6591,485.6137,009__2__495,7__2016
8,2016,009,2,495,300.1430,310.0502,222.3963,172.2356,203.1352,⋯,209.6567,258.2191,218.2689,147.0546,127.4533,275.3076,310.5383,245.7551,009__2__495,8__2016
6,2016,010,2,495,52477.3520,36470.9531,37612.5839,31001.7163,28577.4171,⋯,49563.1946,37982.5034,55510.9617,44398.8307,32484.5081,37905.9025,38919.1104,47850.1981,010__2__495,6__2016
7,2016,010,2,495,642.6258,488.3443,899.0803,429.0197,349.5420,⋯,657.8713,670.5887,697.3044,840.3402,688.7687,401.1185,570.9033,857.5402,010__2__495,7__2016
8,2016,010,2,495,676.3947,360.4272,155.5763,157.4829,183.5319,⋯,351.1151,263.3309,386.3082,318.1149,189.6787,194.9978,166.5396,251.0664,010__2__495,8__2016


In [11]:
## Get the var names to be able to melt later on
var_melters <- c(paste(not_corring_over, collapse= "__"), paste(corring_over, collapse= "__"))

In [12]:
## Melt the draws and add a column called "var_name"
dalys_ihd_long <- melt(dalys_ihd, id.vars = c(var_melters), 
                       measure.vars = paste0("draw_", c(0:999)),
                      value.name = "data", variable.name = "draw_num")

dalys_diabetes_long <- melt(dalys_diabetes, id.vars =c(var_melters), 
                            measure.vars = paste0("draw_", c(0:999)),
                            value.name = "data", variable.name = "draw_num")

In [13]:
## Bind the dataframes
dalys_binded_long <- rbind(dalys_diabetes_long, dalys_ihd_long)

## (2) Devising AR correlation matrix for each unique corring variable (age, sex and cause) and then combine... maybe?

#### TO DO: Find a way to parse the variables and groups into default kron matrices. I think Maddie's function already does that..?

In [14]:
## Number of groups
age_groups <- unique(dalys_ihd[year_id == 2016, age_group_id])
sex_groups <- unique(dalys_ihd[year_id == 2016, sex_id])
year_groups <- unique(dalys_ihd[year_id == 2016, year_id])
loc_groups <- unique(dalys_ihd[year_id == 2016, location_id])
var_groups <- unique(c("heart_stuff", "diabeetus"))

In [15]:
## Matrices within each group first (2x2 matrices do not follow AR processes)
age_corr_mat <- 0.75**abs(outer(1:length(age_groups), 1:length(age_groups), "-"))
colnames(age_corr_mat) = rownames(age_corr_mat) = age_groups

sex_corr_mat <- 0.4**abs(outer(1:length(sex_groups), 1:length(sex_groups), "-"))
colnames(sex_corr_mat) = rownames(sex_corr_mat) = sex_groups

var_corr_mat <- 0.6**abs(outer(1:length(var_groups), 1:length(var_groups), "-"))
colnames(var_corr_mat) = rownames(var_corr_mat) = var_groups

In [16]:
## Create the ultimate correlation matrix: the Kronecker product (IN THE REVERSE ORDER OF age_sex_var)
k1 <- kronecker(var_corr_mat, sex_corr_mat, make.dimnames = T)
k2 <- kronecker(k1, age_corr_mat, make.dimnames = T)

In [17]:
## The time has come: make the data.table into a multi dimensional array!
system.time(dalys_array <- reshape2::acast(dalys_binded_long, get(var_melters[1]) ~ get(var_melters[2]) ~ draw_num, 
                                           value.var = "data"))
str(dalys_array)

   user  system elapsed 
  0.849   0.091   1.026 

 num [1:21, 1:20, 1:1000] 10356 13329 11209 6807 6566 ...
 - attr(*, "dimnames")=List of 3
  ..$ : chr [1:21] "6__1990" "6__1995" "6__2000" "6__2005" ...
  ..$ : chr [1:20] "006__1__495" "006__1__587" "006__2__495" "006__2__587" ...
  ..$ : chr [1:1000] "draw_0" "draw_1" "draw_2" "draw_3" ...


In [18]:
## Copulate all over my body
system.time(dalys_corr <- draw2Dcopula(X = dalys_array, cor_mat = k2, df_return = T))
names(dalys_corr) <- colnames(dalys_binded_long)
str(dalys_corr)

   user  system elapsed 
  0.655   0.028   0.735 

Classes ‘data.table’ and 'data.frame':	420000 obs. of  4 variables:
 $ location_id__year_id          : Factor w/ 21 levels "6__1990","6__1995",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ age_group_id__sex_id__cause_id: Factor w/ 20 levels "006__1__495",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ draw_num                      : Factor w/ 1000 levels "draw_0","draw_1",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ data                          : num  8120 10103 8308 5012 4740 ...
 - attr(*, ".internal.selfref")=<externalptr> 


### Splitting the stacked variables

In [20]:
dalys_correlated <- unstack_df(data = dalys_corr, corr_unstack = corring_over, no_corr_unstack = not_corring_over)

In [22]:
head(dalys_correlated)

location_id,year_id,age_group_id,sex_id,cause_id,draw_num,data
6,1990,6,1,495,draw_0,8119.710
6,1995,6,1,495,draw_0,10102.812
6,2000,6,1,495,draw_0,8307.640
6,2005,6,1,495,draw_0,5012.243
6,2006,6,1,495,draw_0,4739.611
6,2010,6,1,495,draw_0,4789.414


In [24]:
## Save out for viz stuff
setwd("/home/j/temp/sadatnfs"); 

Loading required package: rhdf5


In [39]:
# Save hdf5
require(rhdf5)
h5createFile(file = "sample_corr.h5")
h5createGroup("sample_corr.h5", "sample_dalys")
h5write(dalys_binded_long, "sample_corr.h5", name = "sample_dalys/uncorrelated")
h5write(dalys_correlated, "sample_corr.h5", name = "sample_dalys/correlated")
H5close()

[1] TRUE

In [56]:
# Save Rdata
save(list = c("dalys_binded_long", "dalys_correlated"), file = "sample_corr.Rdata")